In [1]:
import torch
import tiktoken

In [2]:
raw_text = "This is my dog Chico. "

In [3]:
tokenizer = tiktoken.get_encoding("gpt2")

In [4]:
enc_text = tokenizer.encode_ordinary( raw_text )
print( enc_text )

[1212, 318, 616, 3290, 609, 3713, 13, 220]


In [7]:
vocab_size = 4
output_dim = 8
inputs = torch.nn.Embedding( vocab_size, output_dim )
print(inputs.weight)

Parameter containing:
tensor([[ 0.2771,  1.2925,  0.4086, -0.2523, -1.1620, -0.6990, -0.6632, -0.5130],
        [-0.8811, -0.1949,  0.0994, -0.8223,  0.9215,  0.1530,  0.4579,  0.1149],
        [-1.8343, -0.3739, -0.5283,  0.7703,  2.1087, -0.9418, -0.4201,  1.8617],
        [ 0.0719, -0.0541,  0.6511, -1.8043,  1.7056,  0.5268,  0.3230,  2.6119]],
       requires_grad=True)


In [8]:
inputs = inputs.weight.data
inputs

tensor([[ 0.2771,  1.2925,  0.4086, -0.2523, -1.1620, -0.6990, -0.6632, -0.5130],
        [-0.8811, -0.1949,  0.0994, -0.8223,  0.9215,  0.1530,  0.4579,  0.1149],
        [-1.8343, -0.3739, -0.5283,  0.7703,  2.1087, -0.9418, -0.4201,  1.8617],
        [ 0.0719, -0.0541,  0.6511, -1.8043,  1.7056,  0.5268,  0.3230,  2.6119]])

In [9]:
inputs.shape

torch.Size([4, 8])

In [10]:
for row in inputs:
    print( row.tolist() )

[0.2770703136920929, 1.2925260066986084, 0.4085632264614105, -0.2523374557495117, -1.1619651317596436, -0.6990135312080383, -0.663209855556488, -0.51301509141922]
[-0.8811240792274475, -0.19490861892700195, 0.09936337918043137, -0.8223011493682861, 0.9215407371520996, 0.15300123393535614, 0.4579039216041565, 0.1148504689335823]
[-1.8342560529708862, -0.37388312816619873, -0.5282573699951172, 0.770287811756134, 2.108671188354492, -0.9418469071388245, -0.42011427879333496, 1.8616602420806885]
[0.07187089323997498, -0.05407668650150299, 0.6511098742485046, -1.8043287992477417, 1.7055691480636597, 0.5267564654350281, 0.3230374753475189, 2.611872911453247]


In [11]:
x = torch.tensor([1.1,2.3])
y = torch.tensor([3.4,-2.1])


In [12]:
torch.dot(x,y)

tensor(-1.0900)

In [13]:
query = inputs[2]
print( query )

tensor([-1.8343, -0.3739, -0.5283,  0.7703,  2.1087, -0.9418, -0.4201,  1.8617])


In [14]:
for i in range(len(inputs)):
    print( torch.dot( query, inputs[i] ) )

tensor(-3.8699)
tensor(2.8237)
tensor(13.3525)
tensor(5.9817)


In [67]:
attention_scores_2 = torch.zeros(len(inputs)) 
for i in range( len( inputs ) ):
    attention_scores_2[i] = torch.dot( query, inputs[i] )
print( attention_scores_2 )

tensor([-3.8699,  2.8237, 13.3525,  5.9817])


In [47]:
# normalize the attention scores using the softmax function
# def softmax(x):
#     torch.exp(x) / torch.exp(x).sum()

In [71]:
attention_weights_2 = torch.softmax(attention_scores_2, dim=0)
attention_weights_2

tensor([3.3120e-08, 2.6738e-05, 9.9934e-01, 6.2890e-04])

In [72]:
attention_weights_2.sum()

tensor(1.0000)

In [ ]:
context_vector_2 = torch.zeros( query.shape )
for i in range( len( attention_weights_2 ) ):
    context_vector_2 += attention_weights_2[i] * inputs[i]
context_vector_2

tensor([-1.8330, -0.3737, -0.5275,  0.7686,  2.1084, -0.9409, -0.4196,  1.8621])

In [74]:
# get all of the attention scores via matrix multiplication
attention_scores_2 = inputs @ inputs.T
attention_scores_2

tensor([[ 4.5198, -1.7883, -3.8699, -3.2329],
        [-1.7883,  2.5959,  2.8237,  3.5959],
        [-3.8699,  2.8237, 13.3525,  5.9817],
        [-3.2329,  3.5959,  5.9817, 13.8003]])

In [76]:
attention_weights = torch.softmax(attention_scores_2, dim=-1)
attention_weights 

tensor([[9.9753e-01, 1.8169e-03, 2.2662e-04, 4.2854e-04],
        [2.5013e-03, 2.0055e-01, 2.5186e-01, 5.4510e-01],
        [3.3120e-08, 2.6738e-05, 9.9934e-01, 6.2890e-04],
        [4.0031e-08, 3.6989e-05, 4.0198e-04, 9.9956e-01]])